In [ ]:
#cargar los datos de sismuestras, antigenicas, PRASS consolidado y PRASS CAMPO

In [1]:
!pip -q install xlsxwriter
# manejo de datos
import pandas as pd
import numpy as np
# regular expressions
import re
# manejo de poligonos
import shapely.wkt
import shapely.speedups
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
# barras de progreso
import tqdm
from tqdm import tqdm
# quitar warnings
import warnings
warnings.filterwarnings('ignore')
tqdm.pandas()

     |████████████████████████████████| 153kB 6.5MB/s 


In [3]:
# Cargar bases de datos de muestras - PCR y ANTIGENAS
sismuestras = '/content/drive/My Drive/Datos/Alcaldia/Analitica/PRASS/DEPARTAMENTO-307-20201130_pcr.csv'
df_sismuestras = pd.read_csv(sismuestras, encoding="UTF-16",sep='|', error_bad_lines=True, delimiter='|')
antigenica = '/content/drive/My Drive/Datos/Alcaldia/Analitica/PRASS/DEPARTAMENTO-307-20201130_antigenica.csv'
df_antigenica = pd.read_csv(antigenica, header=1)

In [4]:
# 1. solo tomar a partir del 22 de agosto 
df_antigenica.fecha = df_antigenica.fecha.astype('datetime64[ns]')
df_antigenica = df_antigenica.loc[df_antigenica.fecha >= '2020-08-22']
# 1. Filtrar PCR por fecha (Mayores a 22 de agosto)
df_sismuestras.FechaMuestra = pd.to_datetime(df_sismuestras.FechaMuestra,  dayfirst=True)
df_sismuestras.Fecha_Resultado = pd.to_datetime(df_sismuestras.Fecha_Resultado,  dayfirst=True)
df_sismuestras = df_sismuestras.loc[df_sismuestras.FechaMuestra>='2020-08-22']

In [5]:
df_sismuestras.Documento = df_sismuestras.Documento.str.strip()
df_antigenica.num_identificacion = df_antigenica.num_identificacion.str.strip() 

In [ ]:
#  PRASS
# Cargar el PRASS consolidado
prass_consolidado = '/content/drive/My Drive/Datos/Alcaldia/Analitica/PRASS/PRASS BUCARAMANGA 2020.xlsx' 
df_prass_consolidado = pd.read_excel(prass_consolidado)

In [ ]:
df_prass_consolidado['Número de documento'] = df_prass_consolidado['Número de documento'].str.strip()

In [ ]:
df_prass_consolidado.groupby('RESULTADO').count()

,ObjectID,GlobalID,CreationDate,Creator,EditDate,Editor,Nombres,Apellidos,Tipo de documento,Otro - Tipo de documento,Número de documento,Fecha de nacimiento,Sexo,Edad,Unidad de medida (Edad),Pertenencia Étnica,Otra - Pertenencia Étnica,¿Se encuentra en estado de embarazo?,Semanas de Gestación,Número de contacto telefónico (celular),Contacto de caso positivo,NO LLENAR,NO LLENAR ESTA PREGUNTA,Muestras y notificación,¿Recibió atención médico-domiciliaria?,¿Fue referido a hospitalización?,Observaciones,Otro - Tipo de documento.1,Otra - Pertenencia Étnica.1,Tratamiento de datos personales,Número de Manzana,Nomenclatura casa o apartamento,Dirección,¿Ha tenido contacto con un caso positivo?,Nombre del contacto positivo,¿Le han realizado una prueba de detección para COVID-19?,¿Qué tipo de prueba le han aplicado?,Fecha de toma de muestra,Resultado de la muestra,Fecha de inicio de síntomas,...,Cáncer,Obesidad,Desnutrición,Enfermedad renal,Tuberculosis,Toma medicamentos inmunosupresores,Fumador,Ninguna,Otra que no esta en la lista,VIH,Fiebre>38,Tos,Dificultad respiratoria,Odinofagia,Dolor de cabeza,Fatiga / Andinamia,Rinorrea,Diarrea,Lesiones en piel,Pérdida del olfato,Pérdida del gusto,Asintomático,Tipo de prueba de hisopado,Muestra de hisopado,Muestra de sangre,Ficha de notificación,La persona no acepta toma de muestra,x,y,NOMBRE,PRIMER APELLIDO,BARRIO,ESTADO VISITA,Fecha Resultado,Dias diferencias,Resultado prueba antigenica,Fecha prueba antigenica,Dias diferencias pba antigenica,Escala tiempo PCR,Escala tiempo antigénica
RESULTADO,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
INDETERMINADO,10,10,10,4,10,4,10,10,10,0,10,10,10,10,10,10,0,4,0,10,0,0,0,0,10,10,0,0,0,10,0,0,10,10,2,10,0,0,0,0,...,0,1,0,1,0,1,1,4,0,0,0,1,0,0,3,0,0,0,0,0,0,6,9,8,0,8,0,10,10,10,10,10,10,10,10,10,0,0,10,0
NEGATIVO,8051,8051,8051,3170,8051,3209,8051,8051,8051,0,8051,8036,8051,8048,8051,8051,0,4738,86,8051,0,84,90,335,7543,7517,757,0,0,8051,0,0,8051,8051,898,7704,636,636,5,1062,...,102,459,28,137,12,217,478,3619,20,26,197,621,173,255,754,240,299,191,24,84,89,6542,8051,6944,13,6293,182,8051,8051,8051,8051,8051,8051,8051,8051,8051,256,170,8051,170
No acepta toma muestra,556,556,556,506,556,541,556,556,556,0,556,551,556,554,556,556,0,356,4,556,0,90,127,556,524,520,21,0,0,556,0,0,556,556,19,109,13,13,0,32,...,9,23,0,10,0,22,23,171,19,0,3,14,7,4,13,4,4,7,1,3,4,401,28,0,0,0,0,556,556,28,16,556,556,16,16,556,12,12,16,12
No aplica,42264,42264,42264,19966,42264,20250,26503,26481,26498,0,26482,26455,26481,26468,26481,26498,0,16846,163,26480,0,569,758,123,25209,25146,1037,44,0,26481,0,0,41939,26481,951,24417,2019,2013,17,531,...,94,362,12,119,16,237,582,21433,83,7,132,281,70,88,322,102,131,93,18,81,92,25193,792,215,5,192,5412,42264,42264,752,0,42264,42264,0,0,42264,752,479,0,479
POSITIVO,1235,1235,1235,637,1235,651,1235,1235,1235,0,1235,1235,1235,1235,1235,1235,0,744,11,1235,0,20,24,66,1159,1151,91,0,0,1235,0,0,1235,1235,214,1168,106,105,0,316,...,23,66,1,18,3,30,46,645,5,5,112,211,65,64,244,100,77,78,4,107,92,810,1235,962,1,872,45,1235,1235,1235,1235,1235,1235,1235,1235,1235,37,30,1235,30
Pendiente,1962,1962,1962,504,1962,507,1962,1962,1962,0,1962,1960,1962,1962,1962,1962,0,1206,28,1962,0,0,0,13,1844,1843,232,0,0,1962,0,0,1962,1962,295,1949,582,588,0,237,...,18,130,5,26,3,38,120,1035,0,4,63,135,34,51,130,49,49,44,4,32,31,1650,1962,1255,0,1164,237,1962,1962,572,564,1962,1962,564,0,1962,49,13,0,13


In [6]:
# cargar PRASS campo
nombre = 'Informe AcumuladoPrass Septembre a Noviembre2020_28.xlsx'
prass_campo_general = '/content/drive/My Drive/Datos/Alcaldia/Analitica/PRASS/'+nombre 
prass_campo_general = pd.read_excel(prass_campo_general, sheet_name='PRASS', header=8)

In [7]:
prass_campo_general

,Item,Fecha de Intervención,Barrio,Primer Nombre,Segundo Nombre,Primer Apellido,Segundo Apellido,Tipo de Documento,Número de documento,Fecha de nacimiento,EPS,Dirección,Teléfono,CLASIFICACIÓN,TIPIFICACIÓN,Prueba Solicitada,Fecha de Toma,Fecha de Reporte,Intervalo (dias),Resultado,Reportado en SIVIGILA,Observacion
0,1,2020-09-02,Kenedy,Milton,Cesar,Badillo,Araujo,CC,2017472,1925-07-04 00:00:00,Suramericana,Calle 15N 10C 15,3016458900-6932850,Comorbilidad,HTA,PCR,2020-09-02 00:00:00,2020-09-04,2.0,NEGATIVO ...,NaN,NaN
1,2,2020-09-02,Kenedy,Yolanda,0,Becerra,Chacon,CC,63291401,1962-01-21 00:00:00,Salud Total,Carrera 10B 15N 04,3197563862-3196350848,Comorbilidad,HTA,PCR,2020-09-02 00:00:00,2020-09-08,6.0,NEGATIVO ...,NaN,NaN
2,3,2020-09-02,Kenedy,Edilia,0,Quitian,Ojeda,CC,37799834,1949-02-16 00:00:00,Nueva Eps,Carrera 10 B 16 N 16,3184250711-6404194,Edad,HTA,PCR,2020-09-02 00:00:00,2020-09-08,6.0,NEGATIVO ...,NaN,NaN
3,4,2020-09-02,Kenedy,Maria,Elena,Talero,Vera,CC,63304310,1962-07-24 00:00:00,Nueva Eps,Calle 16An 10D 26,3163286227-3138187003,Comorbilidad,HTA,PCR,2020-09-02 00:00:00,2020-09-08,6.0,NEGATIVO ...,NaN,NaN
4,5,2020-09-02,Kenedy,Gonzalo,0,Mancilla,Acevedo,CC,91206626,1957-01-01 00:00:00,Nueva Eps,Calle 16An 10D 26,3138187003-3163286227,Edad,NaN,PCR,2020-09-02 00:00:00,2020-09-08,6.0,NEGATIVO ...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8187,8188,2020-11-28,Chapinero,José,Miguel,Cruz,Duran,CC,91264678,1969-05-01 00:00:00,Coomeva,Carrera 16 4 57,6718494,Comorbilidad,Fumador,PCR,2020-11-28 00:00:00,NaT,NaN,NaN,NaN,NaN
8188,8189,2020-11-28,Chapinero,Viviana,NaN,Gutierrez,NaN,CC,1098619982,1986-07-27 00:00:00,Sanitas,Carrera 15B 3 29,3165204132,Alta Movilidad,NaN,PCR,2020-11-28 00:00:00,NaT,NaN,NaN,NaN,NaN
8189,8190,2020-11-28,Chapinero,Luis,Alejandro,Ordonez,Ordoñez,CC,1095839162,1998-10-14 00:00:00,Sanitas,Carrera 15B 3 29,3163223422,Alta Movilidad,NaN,PCR,2020-11-28 00:00:00,NaT,NaN,NaN,NaN,NaN
8190,8191,2020-11-28,Chapinero,Alejandro,NaN,Hernandez,Rodriguez,CC,91265428,1969-05-05 00:00:00,Sanitas,Calle 4 16 13,3168797714,Alta Movilidad,NaN,PCR,2020-11-28 00:00:00,NaT,NaN,NaN,NaN,NaN


In [8]:
prass_campo_general['Número de documento'] = prass_campo_general['Número de documento'].astype(str).str.strip() 

In [9]:
prass_campo_general['Número de documento'].shape

(8192,)

In [10]:
df_prass_campo = prass_campo_general

In [11]:
df_prass_campo

,Item,Fecha de Intervención,Barrio,Primer Nombre,Segundo Nombre,Primer Apellido,Segundo Apellido,Tipo de Documento,Número de documento,Fecha de nacimiento,EPS,Dirección,Teléfono,CLASIFICACIÓN,TIPIFICACIÓN,Prueba Solicitada,Fecha de Toma,Fecha de Reporte,Intervalo (dias),Resultado,Reportado en SIVIGILA,Observacion
0,1,2020-09-02,Kenedy,Milton,Cesar,Badillo,Araujo,CC,2017472,1925-07-04 00:00:00,Suramericana,Calle 15N 10C 15,3016458900-6932850,Comorbilidad,HTA,PCR,2020-09-02 00:00:00,2020-09-04,2.0,NEGATIVO ...,NaN,NaN
1,2,2020-09-02,Kenedy,Yolanda,0,Becerra,Chacon,CC,63291401,1962-01-21 00:00:00,Salud Total,Carrera 10B 15N 04,3197563862-3196350848,Comorbilidad,HTA,PCR,2020-09-02 00:00:00,2020-09-08,6.0,NEGATIVO ...,NaN,NaN
2,3,2020-09-02,Kenedy,Edilia,0,Quitian,Ojeda,CC,37799834,1949-02-16 00:00:00,Nueva Eps,Carrera 10 B 16 N 16,3184250711-6404194,Edad,HTA,PCR,2020-09-02 00:00:00,2020-09-08,6.0,NEGATIVO ...,NaN,NaN
3,4,2020-09-02,Kenedy,Maria,Elena,Talero,Vera,CC,63304310,1962-07-24 00:00:00,Nueva Eps,Calle 16An 10D 26,3163286227-3138187003,Comorbilidad,HTA,PCR,2020-09-02 00:00:00,2020-09-08,6.0,NEGATIVO ...,NaN,NaN
4,5,2020-09-02,Kenedy,Gonzalo,0,Mancilla,Acevedo,CC,91206626,1957-01-01 00:00:00,Nueva Eps,Calle 16An 10D 26,3138187003-3163286227,Edad,NaN,PCR,2020-09-02 00:00:00,2020-09-08,6.0,NEGATIVO ...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8187,8188,2020-11-28,Chapinero,José,Miguel,Cruz,Duran,CC,91264678,1969-05-01 00:00:00,Coomeva,Carrera 16 4 57,6718494,Comorbilidad,Fumador,PCR,2020-11-28 00:00:00,NaT,NaN,NaN,NaN,NaN
8188,8189,2020-11-28,Chapinero,Viviana,NaN,Gutierrez,NaN,CC,1098619982,1986-07-27 00:00:00,Sanitas,Carrera 15B 3 29,3165204132,Alta Movilidad,NaN,PCR,2020-11-28 00:00:00,NaT,NaN,NaN,NaN,NaN
8189,8190,2020-11-28,Chapinero,Luis,Alejandro,Ordonez,Ordoñez,CC,1095839162,1998-10-14 00:00:00,Sanitas,Carrera 15B 3 29,3163223422,Alta Movilidad,NaN,PCR,2020-11-28 00:00:00,NaT,NaN,NaN,NaN,NaN
8190,8191,2020-11-28,Chapinero,Alejandro,NaN,Hernandez,Rodriguez,CC,91265428,1969-05-05 00:00:00,Sanitas,Calle 4 16 13,3168797714,Alta Movilidad,NaN,PCR,2020-11-28 00:00:00,NaT,NaN,NaN,NaN,NaN


In [12]:
# cruce sismuestras prass campo
cruce_PCA = df_sismuestras.loc[df_sismuestras.Documento.isin(df_prass_campo['Número de documento'].astype(str)), ['Documento', 
                                                                                                      'Fecha_Resultado',
                                                                                                      'Resultado',
                                                                                                      'FechaMuestra']]
cruce_PCA['fuente'] = 'sismuestras'                                                                                                       

In [13]:
# cruce con antigenicas
cruce_PCB = df_antigenica.loc[df_antigenica.num_identificacion.isin(df_prass_campo['Número de documento'].astype(str)), ['num_identificacion',
                                                                                                             'fecha',
                                                                                                             'resultado']]                                                                                                            

In [14]:
# renombrar las columnas de antigenicas
cruce_PCB.rename(columns={'num_identificacion':'Documento',
                          'fecha':'Fecha_Resultado',
                          'resultado':'Resultado'}, inplace=True)
cruce_PCB['fuente'] = 'antigena'

In [15]:
concat_muestas = pd.concat([cruce_PCA,cruce_PCB])
concat_muestas

,Documento,Fecha_Resultado,Resultado,FechaMuestra,fuente
71388,91494573,2020-08-23,POSITIVO ...,2020-08-22,sismuestras
74025,28213044,2020-08-25,POSITIVO ...,2020-08-22,sismuestras
77070,1095910349,2020-08-27,NEGATIVO ...,2020-08-24,sismuestras
79477,1099364328,2020-08-29,POSITIVO ...,2020-08-26,sismuestras
79860,1098664528,2020-08-29,POSITIVO ...,2020-08-27,sismuestras
...,...,...,...,...,...
62181,1098671867,2020-10-20,NEGATIVO,NaT,antigena
62601,1098700520,2020-11-05,NEGATIVO,NaT,antigena
62867,1098658592,2020-10-07,NEGATIVO,NaT,antigena
63546,1005109569,2020-10-26,NEGATIVO,NaT,antigena


In [16]:
cc_duplicadas = concat_muestas.Documento[concat_muestas.Documento.duplicated()]
muestras_parte_1 = concat_muestas.loc[~concat_muestas.Documento.isin(cc_duplicadas)]
muestras_parte_2 = concat_muestas.loc[concat_muestas.Documento.isin(cc_duplicadas)]
muestras_parte_1.shape, muestras_parte_2.shape

((6760, 5), (872, 5))

In [17]:
# antes de actualizar quitar duplicados del prass campo
cc_duplicadas_prass_campo = df_prass_campo['Número de documento'][df_prass_campo['Número de documento'].duplicated()]
df_prass_campo_parte_1 = df_prass_campo.loc[~df_prass_campo['Número de documento'].isin(cc_duplicadas_prass_campo)]

# la parte 2 son los repetidos + los que no se encontraron en muestras parte 1
df_prass_campo_parte_2 = df_prass_campo.loc[(df_prass_campo['Número de documento'].isin(cc_duplicadas_prass_campo))
                                            |(~df_prass_campo['Número de documento'].isin(muestras_parte_1['Documento']))]

In [18]:
# actualizar la primera tabla de prass campo - los no repetidos
df_prass_campo_parte_1 = df_prass_campo_parte_1.merge(muestras_parte_1,left_on='Número de documento', right_on='Documento')

In [19]:
# calcular la demora
df_prass_campo_parte_1['demora'] = (df_prass_campo_parte_1.Fecha_Resultado.astype('datetime64[ns]') - df_prass_campo_parte_1['Fecha de Toma'].astype('datetime64[ns]')).dt.days

In [20]:
# los que tienen el resultado antes de la toma van a vacio
df_prass_campo_parte_1.loc[df_prass_campo_parte_1.demora<0, ['Fecha_Resultado', 'Resultado_y', 'fuente', 'demora']] = None

In [21]:
df_prass_campo_parte_1

,Item,Fecha de Intervención,Barrio,Primer Nombre,Segundo Nombre,Primer Apellido,Segundo Apellido,Tipo de Documento,Número de documento,Fecha de nacimiento,EPS,Dirección,Teléfono,CLASIFICACIÓN,TIPIFICACIÓN,Prueba Solicitada,Fecha de Toma,Fecha de Reporte,Intervalo (dias),Resultado_x,Reportado en SIVIGILA,Observacion,Documento,Fecha_Resultado,Resultado_y,FechaMuestra,fuente,demora
0,1,2020-09-02,Kenedy,Milton,Cesar,Badillo,Araujo,CC,2017472,1925-07-04 00:00:00,Suramericana,Calle 15N 10C 15,3016458900-6932850,Comorbilidad,HTA,PCR,2020-09-02 00:00:00,2020-09-04,2.0,NEGATIVO ...,NaN,NaN,2017472,2020-09-04,NEGATIVO ...,2020-09-03,sismuestras,2.0
1,2,2020-09-02,Kenedy,Yolanda,0,Becerra,Chacon,CC,63291401,1962-01-21 00:00:00,Salud Total,Carrera 10B 15N 04,3197563862-3196350848,Comorbilidad,HTA,PCR,2020-09-02 00:00:00,2020-09-08,6.0,NEGATIVO ...,NaN,NaN,63291401,2020-09-08,NEGATIVO ...,2020-09-04,sismuestras,6.0
2,3,2020-09-02,Kenedy,Edilia,0,Quitian,Ojeda,CC,37799834,1949-02-16 00:00:00,Nueva Eps,Carrera 10 B 16 N 16,3184250711-6404194,Edad,HTA,PCR,2020-09-02 00:00:00,2020-09-08,6.0,NEGATIVO ...,NaN,NaN,37799834,2020-09-08,NEGATIVO ...,2020-09-03,sismuestras,6.0
3,4,2020-09-02,Kenedy,Maria,Elena,Talero,Vera,CC,63304310,1962-07-24 00:00:00,Nueva Eps,Calle 16An 10D 26,3163286227-3138187003,Comorbilidad,HTA,PCR,2020-09-02 00:00:00,2020-09-08,6.0,NEGATIVO ...,NaN,NaN,63304310,2020-09-08,NEGATIVO ...,2020-09-03,sismuestras,6.0
4,5,2020-09-02,Kenedy,Gonzalo,0,Mancilla,Acevedo,CC,91206626,1957-01-01 00:00:00,Nueva Eps,Calle 16An 10D 26,3138187003-3163286227,Edad,NaN,PCR,2020-09-02 00:00:00,2020-09-08,6.0,NEGATIVO ...,NaN,NaN,91206626,2020-09-08,NEGATIVO ...,2020-09-03,sismuestras,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6752,8122,2020-11-28,Chapinero,Silvia,Alejandra,Rivera,Arenas,CC,1005329805,2002-10-21 00:00:00,Salud Total,Calle 3 15 63 Torre 2 Apto 702,3152580218,Sintomático,NaN,PCR,2020-11-28 00:00:00,NaT,NaN,NaN,NaN,NaN,1005329805,NaT,None,2020-09-04,None,NaN
6753,8124,2020-11-28,Chapinero,Maria,Alejandra,Soto,Hernandez,CC,1127916705,1994-04-11 00:00:00,Nueva Eps,Calle 3 15 63 Torre 2 Apto 305,3003998618,Sintomático,NaN,PCR,2020-11-28 00:00:00,NaT,NaN,NaN,NaN,NaN,1127916705,2020-11-28,POSITIVO,NaT,antigena,0.0
6754,8129,2020-11-28,Chapinero,Mariana,NaN,Cubides,Rojas,TI,1097117833,2012-10-25 00:00:00,Coomeva,Calle 3 15 63 Torre 2 Apto 905,3232930040,Contacto estrecho caso confirmado,Asma,PCR,2020-11-28 00:00:00,NaT,NaN,NaN,NaN,NaN,1097117833,NaT,None,2020-10-30,None,NaN
6755,8148,2020-11-28,Chapinero,Jeferson,NaN,Cubides,Herrera,CC,1095814301,1992-12-15 00:00:00,Coomeva,Calle 3 15 63 Torre 2 Apto 905,3232930040,Sintomático,NaN,PCR,2020-11-28 00:00:00,NaT,NaN,NaN,NaN,NaN,1095814301,NaT,None,2020-11-12,None,NaN


In [22]:
# comprobar que si esten todos los registros de prass campos
assert df_prass_campo_parte_1.shape[0] + df_prass_campo_parte_2.shape[0] == df_prass_campo.shape[0], 'Deberia ser la misma cantidad de registros'

In [23]:
# actualizacion parte 2 o (no encontrados o repetidos)
df_prass_campo_parte_2

,Item,Fecha de Intervención,Barrio,Primer Nombre,Segundo Nombre,Primer Apellido,Segundo Apellido,Tipo de Documento,Número de documento,Fecha de nacimiento,EPS,Dirección,Teléfono,CLASIFICACIÓN,TIPIFICACIÓN,Prueba Solicitada,Fecha de Toma,Fecha de Reporte,Intervalo (dias),Resultado,Reportado en SIVIGILA,Observacion
5,6,2020-09-02,Kenedy,Norberto,0,Sanchez,Diaz,CC,13806885,1949-06-06 00:00:00,Sanitas,Calle 16 Bn 10D 33,3174145016- 6737784,Edad,NaN,PCR,2020-09-02 00:00:00,2020-09-07,5.0,NEGATIVO ...,NaN,NaN
29,30,2020-09-03,Kenedy,Vivian,Mayerly,Echeverry,Mogollon,CC,63538975,1982-10-16 00:00:00,Nueva Eps,Calle 16 8 11 Tejar 1,6737736,Sintomático,NaN,PCR,2020-09-03 00:00:00,NaT,-44077.0,NaN,SI,NaN
30,31,2020-09-03,Kenedy,Martha,Beatriz,Mogollon,Mantilla,CC,63281152,1960-05-04 00:00:00,Nueva Eps,Calle 16 8 11,6737736,Edad,NaN,PCR,2020-09-03 00:00:00,2020-09-08,5.0,NEGATIVO ...,SI,NaN
34,35,2020-09-03,Kenedy,Ismael,0,Reategui,Moreno,CC,13838681,1957-07-30 00:00:00,Salud Total,Calle 15N 8 29 Piso 2,3185291894-6730071,Edad,Cáncer,PCR,2020-09-03 00:00:00,2020-09-08,5.0,POSITIVO ...,SI,NaN
35,36,2020-09-03,Kenedy,Beatriz,0,Cristancho,Sanguino,CC,63353668,1969-02-22 00:00:00,Salud Total,Calle 15N 8 29 Piso 2,3185291894-6730071,Comorbilidad,HTA,PCR,2020-09-03 00:00:00,2020-09-08,5.0,POSITIVO ...,SI,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8187,8188,2020-11-28,Chapinero,José,Miguel,Cruz,Duran,CC,91264678,1969-05-01 00:00:00,Coomeva,Carrera 16 4 57,6718494,Comorbilidad,Fumador,PCR,2020-11-28 00:00:00,NaT,NaN,NaN,NaN,NaN
8188,8189,2020-11-28,Chapinero,Viviana,NaN,Gutierrez,NaN,CC,1098619982,1986-07-27 00:00:00,Sanitas,Carrera 15B 3 29,3165204132,Alta Movilidad,NaN,PCR,2020-11-28 00:00:00,NaT,NaN,NaN,NaN,NaN
8189,8190,2020-11-28,Chapinero,Luis,Alejandro,Ordonez,Ordoñez,CC,1095839162,1998-10-14 00:00:00,Sanitas,Carrera 15B 3 29,3163223422,Alta Movilidad,NaN,PCR,2020-11-28 00:00:00,NaT,NaN,NaN,NaN,NaN
8190,8191,2020-11-28,Chapinero,Alejandro,NaN,Hernandez,Rodriguez,CC,91265428,1969-05-05 00:00:00,Sanitas,Calle 4 16 13,3168797714,Alta Movilidad,NaN,PCR,2020-11-28 00:00:00,NaT,NaN,NaN,NaN,NaN


In [24]:
items = df_prass_campo_parte_2.groupby(['Número de documento']).count()['Item']

In [25]:
df_prass_campo_parte_2['Fecha_Resultado'] = None 
df_prass_campo_parte_2['Resultado_y'] = None 
df_prass_campo_parte_2['fuente'] = None 
df_prass_campo_parte_2['demora'] = None 
df_prass_campo_parte_2['Documento'] = None 
df_prass_campo_parte_2['FechaMuestra'] = None

In [26]:
def update(df_prass_campo_parte_2, cc_2, result, type_):
    # 2. seleccionar la de menor tiempo
    f = df_prass_campo_parte_2.loc[df_prass_campo_parte_2['Número de documento']==cc_2]['Fecha de Toma']
    dias = np.inf
    print('Cedula procesada {}'.format(cc_2))
    for ind, row in result.iterrows():
        demora = (row['Fecha_Resultado']-f.astype('datetime64[ns]')).dt.days.values[0]
        # tomo el resultado de la muestra que coincida exactamente
        if type_=='PCR' and (f.astype('datetime64[ns]')==row['FechaMuestra']).values[0]:
            dias = demora
            # actualizar los valores
            df_prass_campo_parte_2.loc[df_prass_campo_parte_2['Número de documento']==cc_2,
                                    'demora'] = demora                     
            df_prass_campo_parte_2.loc[df_prass_campo_parte_2['Número de documento']==cc_2,
                                    'Fecha_Resultado'] = row['Fecha_Resultado']
            df_prass_campo_parte_2.loc[df_prass_campo_parte_2['Número de documento']==cc_2,
                                    'fuente'] = fuentes[0] 
            df_prass_campo_parte_2.loc[df_prass_campo_parte_2['Número de documento']==cc_2,
                                    'Resultado_y'] = row['Resultado']
            df_prass_campo_parte_2.loc[df_prass_campo_parte_2['Número de documento']==cc_2,
                                    'Documento'] = cc_2 
            df_prass_campo_parte_2.loc[df_prass_campo_parte_2['Número de documento']==cc_2,
                                    'FechaMuestra'] = row['FechaMuestra']                                            
        # si no tomo la menor
        elif demora > 0  and demora < dias:
            dias = demora
            # actualizar los valores
            df_prass_campo_parte_2.loc[df_prass_campo_parte_2['Número de documento']==cc_2,
                                    'demora'] = demora                     
            df_prass_campo_parte_2.loc[df_prass_campo_parte_2['Número de documento']==cc_2,
                                    'Fecha_Resultado'] = row['Fecha_Resultado']
            df_prass_campo_parte_2.loc[df_prass_campo_parte_2['Número de documento']==cc_2,
                                    'fuente'] = fuentes[0] 
            df_prass_campo_parte_2.loc[df_prass_campo_parte_2['Número de documento']==cc_2,
                                    'Resultado_y'] = row['Resultado']
            df_prass_campo_parte_2.loc[df_prass_campo_parte_2['Número de documento']==cc_2,
                                    'Documento'] = cc_2   
            df_prass_campo_parte_2.loc[df_prass_campo_parte_2['Número de documento']==cc_2,
                                    'FechaMuestra'] = row['FechaMuestra']                                                                     
    return  df_prass_campo_parte_2                               

In [27]:
np.unique(items.values, return_counts=True)

(array([1, 2]), array([1423,    6]))

In [28]:
# encontrar la distribucion de repetidos en las muestras
cc_duplicados_distribucion_1 = dict()
for cc_2, rep in zip(items.index, items.values):
    times = muestras_parte_2[muestras_parte_2.Documento.isin([cc_2])].shape[0]
    if rep == 1:
        # encontro registro
        if times != 0:
            # 1 si todas son de un mismo tipo
            result = muestras_parte_2[muestras_parte_2.Documento.isin([cc_2])]
            fuentes = list(result['fuente'].unique())
            # tipo de prueba PCR con sismuestra
            prueba = df_prass_campo_parte_2.loc[df_prass_campo_parte_2['Número de documento']==cc_2]['Prueba Solicitada'] 
            if len(fuentes)==1:
                if fuentes[0] == 'sismuestras' and prueba.str.strip().values[0]=='PCR':
                    df_prass_campo_parte_2 = update(df_prass_campo_parte_2, cc_2, result, type_='PCR')
                elif fuentes[0] == 'antigena' and prueba.str.strip().values[0]!='PCR':
                    df_prass_campo_parte_2 = update(df_prass_campo_parte_2, cc_2, result, type_='ANT')                                     
            else:
                # filtrar las que sean PCR con sismuestras y luego tomar la de menor tiempo
                if prueba.str.strip().values[0]=='PCR':
                    df_prass_campo_parte_2 = update(df_prass_campo_parte_2, cc_2, result.loc[result.fuente=='sismuestras'], type_='PCR')
                else:
                    df_prass_campo_parte_2 = update(df_prass_campo_parte_2, cc_2, result.loc[result.fuente=='antigena'], type_='ANT') 
    # los que tienen dos pruebas
    elif rep ==2:
        # encontro registro
        if times != 0:
            pass # To Do                                                                                                                                                         

Cedula procesada 1005105852
Cedula procesada 1005109569
Cedula procesada 1005235436
Cedula procesada 1005259128
Cedula procesada 1005321738
Cedula procesada 1005330158
Cedula procesada 1005333733
Cedula procesada 1005340396
Cedula procesada 10267961
Cedula procesada 1030553441
Cedula procesada 1045716549
Cedula procesada 1061742246
Cedula procesada 1063617478
Cedula procesada 1091134151
Cedula procesada 1095309690
Cedula procesada 1095791091
Cedula procesada 1095796023
Cedula procesada 1095802293
Cedula procesada 1095819288
Cedula procesada 1095829060
Cedula procesada 1095829242
Cedula procesada 1095835000
Cedula procesada 1095910349
Cedula procesada 1095910433
Cedula procesada 1095918218
Cedula procesada 1095932866
Cedula procesada 1095939181
Cedula procesada 1095940893
Cedula procesada 1095947426
Cedula procesada 1095949610
Cedula procesada 1095954841
Cedula procesada 1096073169
Cedula procesada 1096199580
Cedula procesada 1096538912
Cedula procesada 1097099404
Cedula procesada 10971

In [29]:
# para tener las mismas variables
df_prass_campo_parte_2.rename(columns={'Resultado':'Resultado_x'}, inplace=True)
df_prass_campo_result = pd.concat([df_prass_campo_parte_1, df_prass_campo_parte_2])
df_prass_campo_result.shape, df_prass_campo.shape

((8192, 28), (8192, 22))

In [30]:
df_prass_campo_result[df_prass_campo_result.Documento == df_prass_campo_result['Número de documento']]

,Item,Fecha de Intervención,Barrio,Primer Nombre,Segundo Nombre,Primer Apellido,Segundo Apellido,Tipo de Documento,Número de documento,Fecha de nacimiento,EPS,Dirección,Teléfono,CLASIFICACIÓN,TIPIFICACIÓN,Prueba Solicitada,Fecha de Toma,Fecha de Reporte,Intervalo (dias),Resultado_x,Reportado en SIVIGILA,Observacion,Documento,Fecha_Resultado,Resultado_y,FechaMuestra,fuente,demora
0,1,2020-09-02,Kenedy,Milton,Cesar,Badillo,Araujo,CC,2017472,1925-07-04 00:00:00,Suramericana,Calle 15N 10C 15,3016458900-6932850,Comorbilidad,HTA,PCR,2020-09-02 00:00:00,2020-09-04,2.0,NEGATIVO ...,NaN,NaN,2017472,2020-09-04 00:00:00,NEGATIVO ...,2020-09-03 00:00:00,sismuestras,2
1,2,2020-09-02,Kenedy,Yolanda,0,Becerra,Chacon,CC,63291401,1962-01-21 00:00:00,Salud Total,Carrera 10B 15N 04,3197563862-3196350848,Comorbilidad,HTA,PCR,2020-09-02 00:00:00,2020-09-08,6.0,NEGATIVO ...,NaN,NaN,63291401,2020-09-08 00:00:00,NEGATIVO ...,2020-09-04 00:00:00,sismuestras,6
2,3,2020-09-02,Kenedy,Edilia,0,Quitian,Ojeda,CC,37799834,1949-02-16 00:00:00,Nueva Eps,Carrera 10 B 16 N 16,3184250711-6404194,Edad,HTA,PCR,2020-09-02 00:00:00,2020-09-08,6.0,NEGATIVO ...,NaN,NaN,37799834,2020-09-08 00:00:00,NEGATIVO ...,2020-09-03 00:00:00,sismuestras,6
3,4,2020-09-02,Kenedy,Maria,Elena,Talero,Vera,CC,63304310,1962-07-24 00:00:00,Nueva Eps,Calle 16An 10D 26,3163286227-3138187003,Comorbilidad,HTA,PCR,2020-09-02 00:00:00,2020-09-08,6.0,NEGATIVO ...,NaN,NaN,63304310,2020-09-08 00:00:00,NEGATIVO ...,2020-09-03 00:00:00,sismuestras,6
4,5,2020-09-02,Kenedy,Gonzalo,0,Mancilla,Acevedo,CC,91206626,1957-01-01 00:00:00,Nueva Eps,Calle 16An 10D 26,3138187003-3163286227,Edad,NaN,PCR,2020-09-02 00:00:00,2020-09-08,6.0,NEGATIVO ...,NaN,NaN,91206626,2020-09-08 00:00:00,NEGATIVO ...,2020-09-03 00:00:00,sismuestras,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7586,7587,2020-11-23,Colseguros Norte,Mabel,Rocío,Flórez,Chaparro,CC,63482692,1973-01-23 00:00:00,Salud Total,Bloque 21 Apartamento 403,3144136245,Alta Movilidad,Personal de Salud,PCR,2020-11-23 00:00:00,2020-11-25,2.0,POSITIVO ...,NaN,NaN,63482692,2020-11-25 00:00:00,POSITIVO ...,2020-11-24 00:00:00,sismuestras,2
7659,7660,2020-11-24,Café Madrid,Lilia,0,Tarazona,Valencia,CC,63309479,1957-12-15 00:00:00,Coosalud,Calle 34An 8A 11 Torre 4 Apto 101,3142820551,Edad,NaN,PCR,2020-11-24 00:00:00,NaT,NaN,NaN,NaN,NaN,63309479,2020-11-28 00:00:00,NEGATIVO ...,2020-11-24 00:00:00,sismuestras,4
7667,7668,2020-11-24,Café Madrid,Jose,Arquimides,Rangel,Lizarazo,CC,2133911,1949-06-19 00:00:00,Coosalud,Calle 33Bn 8 14,3174727470,Edad,NaN,PCR,2020-11-24 00:00:00,NaT,NaN,NaN,NaN,NaN,2133911,2020-11-28 00:00:00,NEGATIVO ...,2020-11-24 00:00:00,sismuestras,4
7670,7671,2020-11-24,Café Madrid,Marino,0,Parada,Rincon,CC,28330637,1948-11-06 00:00:00,Comparta,Calle 35 8N 38,6407247,Edad,Tendero,PCR,2020-11-24 00:00:00,NaT,NaN,NaN,NaN,NaN,28330637,2020-11-29 00:00:00,NEGATIVO ...,2020-11-24 00:00:00,sismuestras,5


In [ ]:
# almacenar como archivos temporales
df_prass_campo_result.to_excel('temporal_'+nombre, index=False)

In [ ]:
# para volver a la estructura original
# 1. Resultado_y --> Resultado_x --> renombrar a Resultado --> eliminar Resultado_y
# 2. Fecha_Resultado --> Fecha de Reporte --> Eliminar Fecha_Resultado 
# 3. demora --> Intervalo (dias) --> Eliminar demora
# 4. Eliminar fuente y documento

In [31]:
#1. 
df_prass_campo_result['Resultado_x'] = df_prass_campo_result['Resultado_y']
df_prass_campo_result.rename(columns={'Resultado_x':'Resultado'}, inplace=True)
del df_prass_campo_result['Resultado_y']

In [32]:
#2.
df_prass_campo_result['Fecha de Reporte'] = df_prass_campo_result['Fecha_Resultado']
del df_prass_campo_result['Fecha_Resultado']

In [33]:
#3.
df_prass_campo_result['Intervalo (dias)'] = df_prass_campo_result['demora']
del df_prass_campo_result['demora']

In [34]:
#4.
del df_prass_campo_result['Documento']
del df_prass_campo_result['fuente']

In [35]:
df_prass_campo_result

,Item,Fecha de Intervención,Barrio,Primer Nombre,Segundo Nombre,Primer Apellido,Segundo Apellido,Tipo de Documento,Número de documento,Fecha de nacimiento,EPS,Dirección,Teléfono,CLASIFICACIÓN,TIPIFICACIÓN,Prueba Solicitada,Fecha de Toma,Fecha de Reporte,Intervalo (dias),Resultado,Reportado en SIVIGILA,Observacion,FechaMuestra
0,1,2020-09-02,Kenedy,Milton,Cesar,Badillo,Araujo,CC,2017472,1925-07-04 00:00:00,Suramericana,Calle 15N 10C 15,3016458900-6932850,Comorbilidad,HTA,PCR,2020-09-02 00:00:00,2020-09-04 00:00:00,2,NEGATIVO ...,NaN,NaN,2020-09-03 00:00:00
1,2,2020-09-02,Kenedy,Yolanda,0,Becerra,Chacon,CC,63291401,1962-01-21 00:00:00,Salud Total,Carrera 10B 15N 04,3197563862-3196350848,Comorbilidad,HTA,PCR,2020-09-02 00:00:00,2020-09-08 00:00:00,6,NEGATIVO ...,NaN,NaN,2020-09-04 00:00:00
2,3,2020-09-02,Kenedy,Edilia,0,Quitian,Ojeda,CC,37799834,1949-02-16 00:00:00,Nueva Eps,Carrera 10 B 16 N 16,3184250711-6404194,Edad,HTA,PCR,2020-09-02 00:00:00,2020-09-08 00:00:00,6,NEGATIVO ...,NaN,NaN,2020-09-03 00:00:00
3,4,2020-09-02,Kenedy,Maria,Elena,Talero,Vera,CC,63304310,1962-07-24 00:00:00,Nueva Eps,Calle 16An 10D 26,3163286227-3138187003,Comorbilidad,HTA,PCR,2020-09-02 00:00:00,2020-09-08 00:00:00,6,NEGATIVO ...,NaN,NaN,2020-09-03 00:00:00
4,5,2020-09-02,Kenedy,Gonzalo,0,Mancilla,Acevedo,CC,91206626,1957-01-01 00:00:00,Nueva Eps,Calle 16An 10D 26,3138187003-3163286227,Edad,NaN,PCR,2020-09-02 00:00:00,2020-09-08 00:00:00,6,NEGATIVO ...,NaN,NaN,2020-09-03 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8187,8188,2020-11-28,Chapinero,José,Miguel,Cruz,Duran,CC,91264678,1969-05-01 00:00:00,Coomeva,Carrera 16 4 57,6718494,Comorbilidad,Fumador,PCR,2020-11-28 00:00:00,None,None,None,NaN,NaN,None
8188,8189,2020-11-28,Chapinero,Viviana,NaN,Gutierrez,NaN,CC,1098619982,1986-07-27 00:00:00,Sanitas,Carrera 15B 3 29,3165204132,Alta Movilidad,NaN,PCR,2020-11-28 00:00:00,None,None,None,NaN,NaN,None
8189,8190,2020-11-28,Chapinero,Luis,Alejandro,Ordonez,Ordoñez,CC,1095839162,1998-10-14 00:00:00,Sanitas,Carrera 15B 3 29,3163223422,Alta Movilidad,NaN,PCR,2020-11-28 00:00:00,None,None,None,NaN,NaN,None
8190,8191,2020-11-28,Chapinero,Alejandro,NaN,Hernandez,Rodriguez,CC,91265428,1969-05-05 00:00:00,Sanitas,Calle 4 16 13,3168797714,Alta Movilidad,NaN,PCR,2020-11-28 00:00:00,None,None,None,NaN,NaN,None


In [37]:
# calcular la cantidad de registros sin pruebas
no_result = df_prass_campo_result[df_prass_campo_result['Resultado'].isna()]
print('sin resultado {} porcentaje {}'.format(no_result.shape[0],no_result.shape[0]/len(df_prass_campo_result)))

sin resultado 1098 porcentaje 0.134033203125


In [52]:
df_prass_campo_result['FechaMuestra'] = df_prass_campo_result['FechaMuestra'].apply(lambda x: pd.to_datetime(x))

In [53]:
df_prass_campo_result['Diferencia fechas muestras'] = (df_prass_campo_result['FechaMuestra'] - df_prass_campo_result['Fecha de Toma'].astype('datetime64[ns]')).dt.days 

In [61]:
df_prass_campo_result.loc[df_prass_campo_result.Resultado.isnull(), ['FechaMuestra', 'Diferencia fechas muestras']] = None

In [62]:
# guardar resultado
df_prass_campo_result.to_excel('resultado_'+nombre, index=False, sheet_name='PRASS')